In [7]:
### Import Functions and Libraries
from Functions import import_commod_data, spread_and_zscore, check_cointegration, generate_signals, backtest, calculate_performance_metrics, plots
import itertools
import pandas as pd


In [8]:
### Function to Commodities Data
# List of all commodities available in yfinance
commodities_list = [
    'ES=F', 'YM=F', 'NQ=F', 'RTY=F', 'ZB=F', 'ZN=F', 'ZF=F', 'ZT=F',
    'GC=F', 'MGC=F', 'SI=F', 'SIL=F', 'PL=F', 'HG=F', 'PA=F', 'CL=F',
    'HO=F', 'NG=F', 'RB=F', 'BZ=F', 'ZC=F', 'ZO=F', 'KE=F',
    'ZR=F', 'ZM=F', 'ZL=F', 'ZS=F', 'GF=F', 'HE=F', 'LE=F', 'CC=F',
    'KC=F', 'CT=F', 'OJ=F', 'SB=F'
]
metals = ['GC=F', 'MGC=F', 'SI=F', 'SIL=F', 'PL=F', 'HG=F', 'PA=F']
energy = ['CL=F', 'HO=F', 'NG=F', 'RB=F', 'BZ=F', 'B0=F']
agriculture = ['ZC=F', 'ZO=F', 'KE=F', 'ZR=F', 'ZM=F', 'ZL=F', 'ZS=F', 'GF=F', 'HE=F', 'LE=F', 'CC=F', 'KC=F', 'CT=F', 'LBS=F', 'OJ=F', 'SB=F']

### Function Call to Load Commodities Data for the Date Range
start_date = '2023-07-01'
end_date = '2024-07-01'
# Download data
data = import_commod_data(commodities_list, start_date, end_date)
#print(data.tail())


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [9]:
### Create list of combinations of all unique pairs
def get_all_pairs(pairs):
    # Generate all unique combinations of pairs
    unique_pairings = list(itertools.combinations(pairs, 2))
    # Convert each tuple into a list
    unique_pairings = [list(pair) for pair in unique_pairings]
    
    return unique_pairings

all_pairs = get_all_pairs(commodities_list)
print('The total number of pairs is',len(all_pairs),'and should be [n*(n-1)]/2 = (35 * 34)/2 = 595')


The total number of pairs is 595 and should be [n*(n-1)]/2 = (35 * 34)/2 = 595


In [18]:
### Compute and Filter based on P-Values (< 0.05)
# Create empty array to store pairs that are cointegrated
cointegrated_pairs = []
# Loop through all pairs
for pairing in all_pairs:
    # Get each pair from each pairing -> if pairing = ['ES=F', 'YM=F'], pair1 = 'ES=F' and pair2 = 'YM=F'
    pair1 = pairing[0]
    pair2 = pairing[1]
    # Check for cointegration
    p_value = check_cointegration(data[pair1], data[pair2])
    # Filter based on p-value
    if p_value < 0.05:
        cointegrated_pairs.append(pairing)

#print(cointegrated_pairs)
coint_percentage = len(cointegrated_pairs) / len(all_pairs) * 100
print(f'The total number of cointegrated pairs is {len(cointegrated_pairs)}, giving a total percentage of {round(coint_percentage, 2)}%')
#print(cointegrated_pairs)


The total number of cointegrated pairs is 43, giving a total percentage of 7.23%


In [11]:
# Create an empty DataFrame to store hedge ratios
results_df = pd.DataFrame(columns = ['Pair1', 'Pair2', 'Spread', 'Z_Score', 'P-Value', 'Returns', 'Cumulative Returns', 'Total Return', 'Percentage Return', 'Annualized Return', 'Sharpe Ratio'])

for pair in cointegrated_pairs:
    # Get each pair from each pairing -> if pairing = ['ES=F', 'YM=F'], pair1 = 'ES=F' and pair2 = 'YM=F'
    pair1 = pair[0]
    pair2 = pair[1]
    # Spread and Z-Score
    spread, z_score = spread_and_zscore(data[pair1], data[pair2])
    # Generate Signals
    signals = generate_signals(spread, z_score, data[pair1], data[pair2])
    # Backtest
    positions, returns, cumulative_rets = backtest(signals, data, pair)
    # Performance Metrics
    total_rets, percentage_rets, ann_rets, sharpe_ratio = calculate_performance_metrics(cumulative_rets, returns)
    # Append results to DataFrame
    result = pd.DataFrame({
        'Pair1': [pair1], 
        'Pair2': [pair2], 
        'Spread': [spread], 
        'Z_Score': [z_score], 
        'P-Value': [round(p_value, 4)], 
        'Returns': [returns], 
        'Cumulative Returns': [cumulative_rets], 
        'Total Return': [round(total_rets, 4)], 
        'Percentage Return': [round(percentage_rets, 4)], 
        'Annualized Return': [round(ann_rets, 4)], 
        'Sharpe Ratio': [round(sharpe_ratio, 4)]
    })
    # Concatenate results to DataFrame
    results_df = pd.concat([results_df, result], ignore_index = True)

results_df.head(5)


,Pair1,Pair2,Spread,Z_Score,P-Value,Returns,Cumulative Returns,Total Return,Percentage Return,Annualized Return,Sharpe Ratio
0,ZB=F,ZN=F,Date 2023-07-03 14.687500 2023-07-05 13....,Date 2023-07-03 2.337368 2023-07-05 2.02...,0.4431,Date 2023-07-03 0.000000 2023-07-05 0.03...,Date 2023-07-03 0.000000 2023-07-05 0.03...,1269.4650,63.4733,64.1173,1.6164
1,ZF=F,CL=F,Date 2023-07-03 37.053749 2023-07-05 34....,Date 2023-07-03 1.731950 2023-07-05 1.35...,0.4431,Date 2023-07-03 0.000000 2023-07-05 0.43...,Date 2023-07-03 0.000000 2023-07-05 0....,27308.3968,1365.4198,1397.2343,2.3595
2,ZF=F,BZ=F,Date 2023-07-03 32.193748 2023-07-05 29....,Date 2023-07-03 1.798923 2023-07-05 1.38...,0.4431,Date 2023-07-03 0.000000 2023-07-05 0.38...,Date 2023-07-03 0.000000 2023-07-05 0....,27044.0037,1352.2002,1383.6201,2.2898
3,ZT=F,CL=F,Date 2023-07-03 31.776405 2023-07-05 29....,Date 2023-07-03 1.737048 2023-07-05 1.36...,0.4431,Date 2023-07-03 0.000000 2023-07-05 0.41...,Date 2023-07-03 0.000000 2023-07-05 0....,42501.3017,2125.0651,2180.9799,2.5361
4,GC=F,MGC=F,Date 2023-07-03 -7.800049 2023-07-05 -7....,Date 2023-07-03 -1.511114 2023-07-05 -1.42...,0.4431,Date 2023-07-03 0.000000 2023-07-05 0.00...,Date 2023-07-03 0.000000 2023-07-05 0.00...,11.6193,0.5810,0.5856,6.8368


In [16]:
### Find Best Pair Based on Highest to Lowest Sharpe Ratio
# Initialize the DataFrame results_df with necessary columns
sorted_df = pd.DataFrame(columns = results_df.columns)
# Assuming results_df is already populated with data at this point
# Sort the DataFrame by 'Sharpe Ratio' in descending order
sorted_df = results_df.sort_values(by = 'Sharpe Ratio', ascending = False)
# Display the top 5 pairs
print(sorted_df.head(10))

# Post Precessing 3 pairs to highlight
# select_pairs = [['BZ=F', 'LE=F'], ['PL=F', 'GF=F'], ['PA=F', 'ZO=F']]

   Pair1  Pair2                                             Spread  \
4   GC=F  MGC=F  Date
2023-07-03    -7.800049
2023-07-05    -7....   
33  BZ=F   LE=F  Date
2023-07-03   -102.174995
2023-07-05    -9...   
18  PL=F   GF=F  Date
2023-07-03    661.399994
2023-07-05    67...   
26  PA=F   ZO=F  Date
2023-07-03    839.150024
2023-07-05    84...   
3   ZT=F   CL=F  Date
2023-07-03    31.776405
2023-07-05    29....   
29  HO=F   ZL=F  Date
2023-07-03   -64.642697
2023-07-05   -66....   
1   ZF=F   CL=F  Date
2023-07-03    37.053749
2023-07-05    34....   
2   ZF=F   BZ=F  Date
2023-07-03    32.193748
2023-07-05    29....   
28  CL=F   BZ=F  Date
2023-07-03   -4.860001
2023-07-05   -4.86...   
14  PL=F   ZR=F  Date
2023-07-03   -895.200012
2023-07-05   -90...   

                                              Z_Score  P-Value  \
4   Date
2023-07-03   -1.511114
2023-07-05   -1.42...   0.4431   
33  Date
2023-07-03   -1.273441
2023-07-05   -0.45...   0.4431   
18  Date
2023-07-03   -0.637903